In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import h3
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from src.settings import *
from shapely.geometry import Polygon
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from src.tools.h3_utils import get_resolution_buffered_suffix
from src.tools.osmnx_utils import generate_data_for_place, get_place_dir_name
from src.tools.feature_extraction import generate_features_for_edges
from pathlib import Path

In [13]:

place_dir = Path(GENERATED_DATA_DIR).joinpath(get_place_dir_name("Berlin,Germany"))
network_type = "drive"
resolution = 9
buffered = True

shp_dir = place_dir.joinpath(f"shp_{network_type}")
edges_shp_path = shp_dir.joinpath(f"edges_hex_{get_resolution_buffered_suffix(resolution, buffered)}.shp")
edges_shp: gpd.GeoDataFrame = gpd.read_file(edges_shp_path)  # type: ignore

# edges_path = place_dir.joinpath(f"edges_{network_type}_{get_resolution_buffered_suffix(resolution, buffered)}.geojson")
# edges: gpd.GeoDataFrame = gpd.read_file(edges_path, driver="GeoJSON")  # type: ignore



In [14]:
for col in edges_shp:
    print(col, edges_shp[col].unique())
edges_shp.columns

id [    0     1     2 ... 42246 42247 42248]
u [    172539     172545     172546 ... 8366394274 8366394275 8533050012]
v [  34694265   34694336     172562 ... 8229138041 8308482803 8533050013]
key [0 1 2 3 4]
osmid ['4610047' '5117633' '[33231778, 4804202, 31740296]' ... '893930330'
 '893930331' '30077245']
name ['Holländerstraße' 'Gotthardstraße' 'Teichstraße' ...
 "['Kaiser-Wilhelm-Platz', 'Kolonnenstraße']" 'Dorndreherweg'
 'Unterbaumstraße']
highway ['tertiary' 'secondary' 'residential' 'primary' 'motorway_link' 'motorway'
 "['motorway', 'motorway_link']" "['secondary', 'motorway_link']"
 'unclassified' "['residential', 'unclassified']" 'secondary_link'
 'primary_link' 'trunk_link' "['motorway', 'primary']" 'living_street'
 "['living_street', 'residential']" "['motorway', 'secondary']"
 "['secondary', 'residential']" 'trunk' "['primary', 'trunk']"
 "['residential', 'tertiary']" 'tertiary_link'
 "['primary_link', 'motorway_link']"
 "['motorway', 'primary_link', 'motorway_link']"
 "[

Index(['id', 'u', 'v', 'key', 'osmid', 'name', 'highway', 'maxspeed', 'oneway',
       'length', 'from', 'to', 'lanes', 'ref', 'bridge', 'tunnel', 'junction',
       'access', 'width', 'service', 'area', 'est_width', 'h3_id', 'geometry'],
      dtype='object')

In [26]:
from src.tools.feature_extraction import convert_to_list

maxspeeds = ['50','30',None,'60',"['80','60']",'80',"['80','100']","['80','50']"
,'100',"['80','50','60']","['30','50']",'40',"['50','70']"
,"['50','60']",'10',"['30','50','70']","['80','40']","['60','40']"
,"['50','40']",'20',"['30','10']","['50','10']",'7','5'
,"['50','60','40']","['20','30']",'70',"['60','70']"
,"['70','50','60']","['30','7']",'DE:urban',"['30','60']"
,"['30','15']","['30','50','60']","['60','50','70']"
,"['30','50','10']","['20','10']",'15']

widths = [None,'12','5','6','8','9','13',"['7','5.5','6.5']"
,"['5.5','7','6.5']",'7','24',"['3','5']",'4.6','3','10.5','14','10'
,'4',"['5','7']","['5','6']",'15','6.5',"['7','6.5']",'5.5'
,"['3','6']",'3.2','7.8',"['4.6','3.5']",'4.5',"['4.5','6.5']",'5.7'
,'12.4','4.7',"['8','6']",'2','12.45','4 m','7.5','11','2.7','4.8','6.2'
,"['3','6','5']",'3.5','11.5','8.5',"['9.4','7','6.5']",'6.7','5.6'
,'6.3',"['5','3']",'25','3.6',"['11','10.5']","['7','6']"
,"['3','3.3']","['5.5','6']",'9.5']

highways = ['tertiary','secondary','residential','primary','motorway_link','motorway'
,"['motorway','motorway_link']","['secondary','motorway_link']"
,'unclassified',"['residential','unclassified']",'secondary_link'
,'primary_link','trunk_link',"['motorway','primary']",'living_street'
,"['living_street','residential']","['motorway','secondary']"
,"['secondary','residential']",'trunk',"['primary','trunk']"
,"['residential','tertiary']",'tertiary_link'
,"['primary_link','motorway_link']"
,"['motorway','primary_link','motorway_link']"
,"['motorway_link','secondary_link']","['secondary','tertiary']"
,"['primary','primary_link']","['living_street','unclassified']"
,"['tertiary_link','tertiary']","['unclassified','tertiary']"
,"['residential','secondary_link']","['secondary','secondary_link']"]

for maxspeed in maxspeeds:
    print(convert_to_list(maxspeed, "maxspeed"))

for width in widths:
    print(convert_to_list(width, "width"))

for highway in highways:
    print(convert_to_list(highway, "highway"))

['50']
['30']
['None']
['60']
['80', '60']
['80']
['80', '100']
['80', '50']
['100']
['80', '50', '60']
['30', '50']
['40']
['50', '70']
['50', '60']
['10']
['30', '50', '70']
['80', '40']
['60', '40']
['50', '40']
['20']
['30', '10']
['50', '10']
['7']
['5']
['50', '60', '40']
['20', '30']
['70']
['60', '70']
['70', '50', '60']
['30', '7']
['None']
['30', '60']
['30', '15']
['30', '50', '60']
['60', '50', '70']
['30', '50', '10']
['20', '10']
['15']
['None']
['12']
['5']
['6']
['8']
['9']
['13']
['7', '6', '6']
['6', '7', '6']
['7']
['24']
['3', '5']
['5']
['3']
['10']
['14']
['10']
['4']
['5', '7']
['5', '6']
['15']
['6']
['7', '6']
['6']
['3', '6']
['3']
['8']
['5', '4']
['4']
['4', '6']
['6']
['12']
['5']
['8', '6']
['2']
['12']
['4']
['8']
['11']
['3']
['5']
['6']
['3', '6', '5']
['4']
['12']
['8']
['9', '7', '6']
['7']
['6']
['6']
['5', '3']
['25']
['4']
['11', '10']
['7', '6']
['3', '3']
['6', '6']
['10']
['tertiary']
['secondary']
['residential']
['primary']
['motorway_link']
[